# Claims Performance Analysis & Pareto by Warehouse

This notebook ingests an exported **dealer claims** spreadsheet, computes cycle times, and produces:
1. **Bucket performance table** — count, mean, median, min, max cycle time (days) by *Bucket Name*.
2. **Pareto of Labels (overall and by warehouse)** — count and cumulative percent, with a Pareto chart for each warehouse.
3. **Power BI–ready outputs** — an Excel file with clean tables for direct import.

> **Cycle time** = `Completed Date − Created Date` (in days).


## Requirements & Assumptions
- Input file is an Excel workbook (e.g., `Claims.xlsx`) with a sheet named **`Tasks`** exported from your tracking system.
- The `Tasks` sheet includes at least these columns:
  - `Bucket Name` (warehouse/location buckets: e.g., OKC, Atlanta, Orlando, Ontario, El Paso, Flowood, Phoenix, Charlotte — plus any admin/problem buckets)
  - `Labels` (category for the event/claim; may be a single label or multiple labels separated by `;`)
  - `Created Date`
  - `Completed Date`
- **Warehouse list** used for *by-warehouse* Pareto is configurable (defaults to your eight sites).

> If some rows do not have a completed date, their cycle time will be `NaN` and excluded from cycle-time aggregations.


In [ ]:
# %%
# Standard imports
from pathlib import Path
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Display options (purely for notebook readability)
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)


In [ ]:
# %%
# ----------------------------
# Parameters (EDIT THESE)
# ----------------------------
# Path to the input claims workbook. By default, looks for 'Claims.xlsx' in the same folder as this notebook.
# If not found, it will try a fallback (used in ChatGPT sandbox), then raise an error.
INPUT_XLSX = Path("Claims.xlsx")
FALLBACK = Path("/mnt/data/Claims.xlsx")  # used only if running in ChatGPT sandbox

# Output Excel for Power BI
OUTPUT_XLSX = Path("Claims_Performance_Summary.xlsx")

# Warehouse names to include in "by warehouse" Pareto (case-sensitive match after stripping)
WAREHOUSES = [
    "OKC", "Atlanta", "Orlando", "Ontario", "El Paso", "Flowood", "Phoenix", "Charlotte"
]

# Maximum labels to show in each Pareto chart for readability (set None to show all)
TOP_N_LABELS = 15


In [ ]:
# %%
# ----------------------------
# Load data
# ----------------------------
def _resolve_input_path() -> Path:
    if INPUT_XLSX.exists():
        return INPUT_XLSX
    if FALLBACK.exists():
        return FALLBACK
    raise FileNotFoundError(f"Could not find input Excel at {INPUT_XLSX} or fallback {FALLBACK}")

in_path = _resolve_input_path()

# Load the 'Tasks' sheet
xls = pd.ExcelFile(in_path)
if "Tasks" not in xls.sheet_names:
    raise ValueError(f"'Tasks' sheet not found. Available sheets: {xls.sheet_names}")

df = pd.read_excel(in_path, sheet_name="Tasks")

# Normalize key columns
for col in ["Bucket Name", "Labels"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()
    else:
        raise ValueError(f"Expected column '{col}' not found in Tasks sheet.")

# Date parsing
for col in ["Created Date", "Completed Date"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")
    else:
        raise ValueError(f"Expected column '{col}' not found in Tasks sheet.")

# Compute cycle time (days)
df["Cycle Time (Days)"] = (df["Completed Date"] - df["Created Date"]).dt.days

# Optional: treat negative durations as NaN if present (uncomment to enforce)
# df.loc[df["Cycle Time (Days)"] < 0, "Cycle Time (Days)"] = np.nan

print(f"Loaded {len(df):,} rows from {in_path.name}")
df.head(3)


## Bucket Performance (Cycle Times by *Bucket Name*)

The table below aggregates cycle time statistics per **Bucket Name**:
- **count** — number of rows with a valid cycle time
- **mean**/**median** — average and median days to completion
- **min**/**max** — shortest and longest observed cycle times

This is exported to the `Bucket_Performance` sheet in the Power BI output workbook.


In [ ]:
# %%
# ----------------------------
# Bucket performance stats
# ----------------------------
bucket_perf = (
    df.groupby("Bucket Name")["Cycle Time (Days)"]
      .agg(["count", "mean", "median", "min", "max"])
      .reset_index()
      .sort_values(["Bucket Name"])
)

# For readability, round floating-point columns (mean/median) to 2 decimals
for c in ["mean", "median"]:
    bucket_perf[c] = bucket_perf[c].round(2)

bucket_perf.rename(columns={
    "count": "Count",
    "mean": "Mean (days)",
    "median": "Median (days)",
    "min": "Min (days)",
    "max": "Max (days)"
}, inplace=True)

bucket_perf.head(10)


## Pareto of Labels — Overall

Counts are computed from the `Labels` column. If multiple labels exist in a single task row separated by `;`, they are split and each counted once.
We then calculate **cumulative percent** in descending order of frequency.

A classic Pareto interpretation: the first ~20% of categories often account for ~80% of occurrences.


In [ ]:
# %%
# ----------------------------
# Pareto - overall Labels
# ----------------------------
labels_df = df[["Bucket Name", "Labels"]].copy()

# Split labels on ';' and explode; strip whitespace and drop empties
labels_exploded = (
    labels_df.assign(Labels=labels_df["Labels"].str.split(";"))
             .explode("Labels")
)
labels_exploded["Labels"] = labels_exploded["Labels"].astype(str).str.strip()
labels_exploded = labels_exploded[labels_exploded["Labels"].ne("")]

# Aggregation overall
labels_overall = (
    labels_exploded["Labels"].value_counts()
    .rename_axis("Label").reset_index(name="Count")
)

total_count = labels_overall["Count"].sum()
labels_overall["Cumulative %"] = labels_overall["Count"].cumsum() / total_count * 100
labels_overall["Cumulative %"] = labels_overall["Cumulative %"].round(2)

labels_overall.head(15)


## Pareto of Labels — By Warehouse

For the *by-warehouse* Pareto, we filter `Bucket Name` to this list (editable in **Parameters** cell):

```
OKC, Atlanta, Orlando, Ontario, El Paso, Flowood, Phoenix, Charlotte
```

For each warehouse, labels are counted and sorted in descending order. We compute within-warehouse **percentage**
and **cumulative percent**, and generate a Pareto chart (Count bars + Cumulative % line).

> To avoid overly wide charts, the parameter `TOP_N_LABELS` limits each chart to the top N labels (default: 15). Set to `None` to show all.


In [ ]:
# %%
# ----------------------------
# Pareto - Labels by Warehouse (Bucket Name)
# ----------------------------
# Keep only the configured warehouses
wh_mask = labels_exploded["Bucket Name"].isin(WAREHOUSES)
labels_by_wh = labels_exploded.loc[wh_mask].copy()
labels_by_wh.rename(columns={"Bucket Name": "Warehouse"}, inplace=True)

# Count labels per warehouse
counts = (
    labels_by_wh.groupby(["Warehouse", "Labels"])
                .size()
                .reset_index(name="Count")
)

# Compute within-warehouse percent and cumulative percent
counts["Percent"] = counts.groupby("Warehouse")["Count"].transform(lambda x: (x / x.sum()) * 100)
counts["Percent"] = counts["Percent"].round(2)

# Sort and compute cumulative % per warehouse
counts = counts.sort_values(["Warehouse", "Count"], ascending=[True, False])
counts["Cumulative %"] = (
    counts.groupby("Warehouse")["Count"]
          .transform(lambda x: x.cumsum() / x.sum() * 100)
          .round(2)
)

# Save a copy for export
labels_by_wh_table = counts.copy()

# ----------------------------
# Plot Pareto for each warehouse
# ----------------------------
def plot_pareto_for_warehouse(sub, warehouse, top_n=15):
    sub_sorted = sub.sort_values("Count", ascending=False)
    if top_n is not None:
        sub_sorted = sub_sorted.head(int(top_n))

    labels = sub_sorted["Labels"].tolist()
    counts_ = sub_sorted["Count"].tolist()
    cum_pct = sub_sorted["Cumulative %"].tolist()

    fig = plt.figure(figsize=(10, 5))  # one chart per figure (no subplots)
    ax1 = plt.gca()
    ax1.bar(labels, counts_)
    ax1.set_xlabel("Label")
    ax1.set_ylabel("Count")
    ax1.set_title(f"Pareto of Labels — {warehouse}")

    # Cumulative percent on a second y-axis
    ax2 = ax1.twinx()
    ax2.plot(range(len(labels)), cum_pct, marker="o")
    ax2.set_ylabel("Cumulative %")
    ax2.set_ylim(0, 100)

    # Rotate x labels for readability
    ax1.tick_params(axis="x", rotation=45, labelsize=9)
    fig.tight_layout()
    plt.show()

# Generate charts
for wh in WAREHOUSES:
    sub = counts[counts["Warehouse"] == wh]
    if not sub.empty:
        plot_pareto_for_warehouse(sub, wh, top_n=TOP_N_LABELS)

# Preview the first few rows of the by-warehouse table
labels_by_wh_table.head(20)


## Export for Power BI

The following tables are written to `Claims_Performance_Summary.xlsx`:
- `Bucket_Performance`
- `Pareto_Labels_Overall`
- `Pareto_Labels_By_Warehouse` (long format: Warehouse, Label, Count, Percent, Cumulative %)

You can import these sheets directly into **Power BI** (Get Data → Excel). Because tables are rebuilt on each run, you can refresh your Power BI model as data updates.


In [ ]:
# %%
# ----------------------------
# Export to Excel for Power BI
# ----------------------------
with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
    bucket_perf.to_excel(writer, sheet_name="Bucket_Performance", index=False)
    labels_overall.to_excel(writer, sheet_name="Pareto_Labels_Overall", index=False)
    labels_by_wh_table.to_excel(writer, sheet_name="Pareto_Labels_By_Warehouse", index=False)

print(f"Exported Power BI workbook -> {OUTPUT_XLSX.resolve()}")


## Next Steps
- Connect `Claims_Performance_Summary.xlsx` to **Power BI** and build visuals from the exported tables.
- Optionally schedule automatic refresh tied to a data export process (e.g., Planner/Service tool → Excel) so your dashboard stays current.
